In [192]:
import json
import openai
import os
import random
from datetime import datetime
from collections import Counter

f = open('/Users/ericmelz/Documents/code/openai-quickstart-python-emelz/.env', 'r')

KEY='OPENAI_API_KEY'
l=f.readline()
api_key = 'undefined!'
while l:
    if l.startswith(KEY):
        print('FOUND')
        api_key = l[len(KEY)+1:-1]
        break
    l = f.readline()

f.close()

openai.api_key=api_key

FOUND


In [3]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"},
    ]
)

In [4]:
response

<OpenAIObject chat.completion id=chatcmpl-8DMzuOHtMtyVc50GRfdKP5nWr8KTb at 0x10423a900> JSON: {
  "id": "chatcmpl-8DMzuOHtMtyVc50GRfdKP5nWr8KTb",
  "object": "chat.completion",
  "created": 1698197358,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The value of the house increased by 150%, so this means there was an increase of 150/100 = $<<150/100*80000=120000>>120,000\nSo the total value of the house was 80,000+50,000 = $<<80000+50000=130000>>130,000\nThis means the value of the house increased to 130,000+120,000 = $<<130000+120000=250000>>250,000\nHis profit was 250,000-130,000 = $<<250000-130000=120000>>120,000. Answer: \\boxed{120000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 139,
    "total_tokens": 195
  }
}

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"},
    ]
)

In [6]:
response

<OpenAIObject chat.completion id=chatcmpl-8DN5fj2F09odoZiSv1qnKS1PqM9Nq at 0x10423a220> JSON: {
  "id": "chatcmpl-8DN5fj2F09odoZiSv1qnKS1PqM9Nq",
  "object": "chat.completion",
  "created": 1698197715,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The increase in value was 80000*1.5=$<<80000*1.5=120000>>120,000\nSo the house is now worth 120000+80000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200000-80000-50000=$<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 96,
    "total_tokens": 152
  }
}

In [5]:
def is_correct(answer):
    return answer.find('\\boxed{70000}') != -1

In [6]:
a1 = "The increase in value was 80000*1.5=$<<80000*1.5=120000>>120,000\nSo the house is now worth 120000+80000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200000-80000-50000=$<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70000}."

In [7]:
a2 = "The value of the house increased by 150%, so this means there was an increase of 150/100 = $<<150/100*80000=120000>>120,000\nSo the total value of the house was 80,000+50,000 = $<<80000+50000=130000>>130,000\nThis means the value of the house increased to 130,000+120,000 = $<<130000+120000=250000>>250,000\nHis profit was 250,000-130,000 = $<<250000-130000=120000>>120,000. Answer: \\boxed{120000}."

In [8]:
is_correct(a1)

True

In [9]:
is_correct(a2)

False

In [10]:
question = "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"

In [11]:
answers = []

## Experiment 1
run the query 100 times, calculate # of good answers, # of bad answers.  Collect all answers and inspect for patterns

In [38]:
def exp1(trials=100):
    for i in range(trials):
        print(f'{i})'.rjust(4, ' '), end='')
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                 "content": question},
            ]
        )
        answer = response.choices[0]['message']['content']
        correct = is_correct(answer)
        if correct:
            print('correct')
        else:
            print('INCORRECT')
        answers.append({'answer': answer,
                        'correct': correct})

In [23]:
exp1()

  0)INCORRECT
  1)correct


In [24]:
answers

[{'answer': 'The value of the house was increased by 80,000*.01=$<<80000*.01=800>>800\nSo the value of the house increased to 50,000+800=$<<50000+800=50800>>50,800\nThat means the value of the house is now 50,800/100=<<50800/100=508>>508\nSo he made a profit of 508*150%=$<<508*150*.01=762.>>762\nSo his profit was 762-50=$<<762-50=712>>712. Answer: \\boxed{712}.',
  'correct': False},
 {'answer': 'The repairs increased the value of the house by 150/100 * $80,000 =$<<150/100*80000=120000>>120,000.\nSo the total value of the house after the repairs are made is $80,000 + $120,000 = $<<80000+120000=200000>>200,000.\nTherefore, Josh made a profit of $200,000 - $80,000 - $50,000 = $<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70000}.',
  'correct': True}]

In [39]:
start = datetime.now()
print(f'Start: {start}')
answers = []
exp1()
end = datetime.now()
print(f'End: {end}')
elapsed = end - start
print(f'Elapsed: {elapsed}')

Start: 2023-10-28 10:33:56.179280
  0)INCORRECT
  1)correct
  2)INCORRECT
  3)correct
  4)INCORRECT
  5)INCORRECT
  6)INCORRECT
  7)INCORRECT
  8)INCORRECT
  9)INCORRECT
 10)correct
 11)correct
 12)INCORRECT
 13)INCORRECT
 14)INCORRECT
 15)correct
 16)INCORRECT
 17)INCORRECT
 18)INCORRECT
 19)correct
 20)INCORRECT
 21)correct
 22)INCORRECT
 23)correct
 24)INCORRECT
 25)INCORRECT
 26)INCORRECT
 27)INCORRECT
 28)correct
 29)correct
 30)correct
 31)INCORRECT
 32)INCORRECT
 33)INCORRECT
 34)INCORRECT
 35)correct
 36)INCORRECT
 37)INCORRECT
 38)correct
 39)INCORRECT
 40)INCORRECT
 41)INCORRECT
 42)correct
 43)correct
 44)INCORRECT
 45)INCORRECT
 46)INCORRECT
 47)INCORRECT
 48)INCORRECT
 49)correct
 50)INCORRECT
 51)INCORRECT
 52)INCORRECT
 53)INCORRECT
 54)INCORRECT
 55)INCORRECT
 56)INCORRECT
 57)INCORRECT
 58)INCORRECT
 59)INCORRECT
 60)INCORRECT
 61)INCORRECT
 62)INCORRECT
 63)INCORRECT
 64)correct
 65)INCORRECT
 66)INCORRECT
 67)INCORRECT
 68)INCORRECT
 69)INCORRECT
 70)INCORRECT
 71)IN

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [40]:
len(answers)

84

In [41]:
n=100-84
n

16

In [42]:
start = datetime.now()
print(f'Start: {start}')
exp1(trials=n)
end = datetime.now()
print(f'End: {end}')
elapsed = end - start
print(f'Elapsed: {elapsed}')

Start: 2023-10-28 10:59:21.898883
  0)INCORRECT
  1)INCORRECT
  2)correct
  3)INCORRECT
  4)correct
  5)INCORRECT
  6)INCORRECT
  7)correct
  8)correct
  9)correct
 10)INCORRECT
 11)INCORRECT
 12)INCORRECT
 13)INCORRECT
 14)INCORRECT
 15)INCORRECT
End: 2023-10-28 11:01:59.838959
Elapsed: 0:02:37.940076


In [43]:
len(answers)

100

In [44]:
exp1_answers = answers

In [47]:
c = Counter()

In [48]:
exp1_answers[0]

{'answer': 'The increase in value was 80000*.01*150=$<<80000*.01*150=120000>>120,000\nSo the total value of the house is 80,000+120,000=$<<80000+120000=200000>>200,000\nThis means he made a profit of 200,000-80,000-50,000=$<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70,000}.',
 'correct': False}

In [52]:
import re

In [67]:
re.match(r'.*\\boxed{70,?000}.*', 'Answer: \\boxed{70000}.')


<re.Match object; span=(0, 22), match='Answer: \\boxed{70000}.'>

In [106]:
def is_correct(answer):
    return re.match(r'.*\\boxed{70,?000}.*', answer, re.DOTALL) is not None

In [107]:
is_correct(exp1_answers[0]['answer'])

True

In [108]:
def update_answer(old_answer):
    return {
        'answer': old_answer['answer'],
        'correct': is_correct(old_answer['answer'])
    }

In [109]:
update_answer(exp1_answers[0])

{'answer': 'The increase in value was 80000*.01*150=$<<80000*.01*150=120000>>120,000\nSo the total value of the house is 80,000+120,000=$<<80000+120000=200000>>200,000\nThis means he made a profit of 200,000-80,000-50,000=$<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70,000}.',
 'correct': True}

In [110]:
updated_answers = [update_answer(a) for a in answers]

In [111]:
old_correct_values = [a['correct'] for a in answers]
old_correct_values[:10]

[False, True, False, True, False, False, False, False, False, False]

In [114]:
updated_correct_values = [a['correct'] for a in updated_answers]
updated_correct_values[:10]

[True, True, False, True, False, False, False, False, False, True]

In [115]:
old_count = Counter(old_correct_values)
old_count

Counter({False: 76, True: 24})

In [116]:
updated_count = Counter(updated_correct_values)
updated_count

Counter({True: 34, False: 66})

In [130]:
def output_exp1_results():
    basedir = 'data/results/exp1'
    os.makedirs('data/results/exp1', exist_ok=True)    
    accuracy = updated_count[True] / (updated_count[True] + updated_count[False])
    with open(f'{basedir}/exp1_master.txt', 'w') as file:
        file.write(f'Accuracy: {accuracy}\n')
        file.write(f'Question: {question}\n\n')
        file.write(f'Correct Answers:\n===============\n')
        for answer in updated_answers:
            if answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')
                
        file.write(f'\n\n\nIncorrect Answers:\n===============\n')
        for answer in updated_answers:
            if not answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')


In [131]:
output_exp1_results()

In [134]:
correct_answers = [a['answer'] for a in updated_answers if a['correct']]

In [135]:
len(correct_answers)

34

In [136]:
correct_answer_set = set(correct_answers)

In [137]:
len(correct_answer_set)

34

In [138]:
correct_answers[0] in correct_answer_set

True

*Note*: There are no duplicates since len(correct_answer_set) == len(correct_answers)

## Experiment2
Add prompt for good answers

In [151]:
def generate_exp2_prompt(n=5):
    preamble = """Given a math problem, generate an answer with a rationale.
    
Examples:
    """
    lines = []
    lines.append(preamble)
    examples = random.sample(correct_answers, n)
    for example in examples:
        lines.append(f'Question: {question}\n')
        lines.append(example)
        lines.append('\n')
    lines.append(f'Question: {question}\n')
    return('\n'.join(lines))
        
    

In [153]:
prompt = generate_exp2_prompt()
print(prompt)

Given a math problem, generate an answer with a rationale.
    
Examples:
    
Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?

The value of the house increased by 80,000*.01*150=<<80000*.01*150=120000>>120,000
So the total price of the house after repairs is 80,000+50,000=$<<80000+50000=130000>>130,000
That means he made a profit of 120,000-50,000 = $<<120000-50000=70000>>70,000. Answer: \boxed{70000}.


Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?

The repairs increased the value by 150/100*80000=$<<150/100*80000=120000>>120000
So the final value is 80000+120000=$<<80000+120000=200000>>200,000
So his profit was 200000-130000=$<<200000-130000=70000>>70,000. Answer: \boxed{70000}.


Question: Josh de

In [154]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", 
         "content": prompt},
    ]
)

In [155]:
response

<OpenAIObject chat.completion id=chatcmpl-8EiuCghmxLhvrWrfnJT30gXPqtY4r at 0x113fd6720> JSON: {
  "id": "chatcmpl-8EiuCghmxLhvrWrfnJT30gXPqtY4r",
  "object": "chat.completion",
  "created": 1698519900,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The value of the house increased by 150/100 * $80,000 = $<<150/100*80000=120000>>120,000.\nSo the total value of the house after repairs is $80,000 + $120,000 = $<<80000+120000=200000>>200,000.\nJosh made a profit of $200,000 - $80,000 - $50,000 = $<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 848,
    "completion_tokens": 114,
    "total_tokens": 962
  }
}

In [167]:
def exp2(trials=100):
    for i in range(trials):
        print(f'{i})'.rjust(4, ' '), end='')
        prompt = generate_exp2_prompt()
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                 "content": prompt},
            ]
        )
        answer = response.choices[0]['message']['content']
        correct = is_correct(answer)
        exp2_answers.append({'answer': answer,
                            'correct': correct})
        if correct:
            print('correct')
        else:
            print('INCORRECT')

In [168]:
exp2_answers = []
start = datetime.now()
print(f'Start: {start}')
exp2()
end = datetime.now()
print(f'End: {end}')
elapsed = end - start
print(f'Elapsed: {elapsed}')

Start: 2023-10-28 12:30:14.948679
  0)correct
  1)INCORRECT
  2)correct
  3)correct
  4)correct
  5)correct
  6)correct
  7)correct
  8)correct
  9)INCORRECT
 10)correct
 11)correct
 12)correct
 13)correct
 14)correct
 15)correct
 16)correct
 17)correct
 18)correct
 19)correct
 20)correct
 21)correct
 22)correct
 23)correct
 24)correct
 25)correct
 26)correct
 27)correct
 28)correct
 29)correct
 30)correct
 31)correct
 32)correct
 33)correct
 34)correct
 35)INCORRECT
 36)correct
 37)correct
 38)correct
 39)correct
 40)INCORRECT
 41)INCORRECT
 42)INCORRECT
 43)correct
 44)correct
 45)correct
 46)correct
 47)correct
 48)correct
 49)correct
 50)INCORRECT
 51)correct
 52)correct
 53)correct
 54)correct
 55)INCORRECT
 56)correct
 57)correct
 58)correct
 59)INCORRECT
 60)correct
 61)INCORRECT
 62)correct
 63)correct
 64)INCORRECT
 65)correct
 66)correct
 67)correct
 68)correct
 69)correct
 70)correct
 71)correct
 72)correct
 73)correct
 74)correct
 75)correct
 76)INCORRECT
 77)INCORRECT
 78)

In [169]:
len(exp2_answers)

100

In [201]:
def output_exp2_results():
    basedir = 'data/results/exp2'
    os.makedirs(basedir, exist_ok=True)    
    correct_counter = Counter([a['correct'] for a in exp2_answers])
    accuracy = correct_counter[True] / (correct_counter[True] + correct_counter[False])
    with open(f'{basedir}/exp2_master.txt', 'w') as file:
        file.write(f'Accuracy: {accuracy}\n')
        file.write(f'Question: {question}\n\n')
        file.write(f'Correct Answers:\n===============\n')
        for answer in exp2_answers:
            if answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')
                
        file.write(f'\n\n\nIncorrect Answers:\n===============\n')
        for answer in exp2_answers:
            if not answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')
                
    with open(f'{basedir}/exp2_correct.jsonl', 'w') as file:
        count = 0
        for answer in exp2_answers:
            if answer['correct']:
                data = {'id': f'exp2_{count}',
                        'question': question,
                        'answer': answer['answer']
                       }
                file.write(json.dumps(data) + '\n')
                count += 1

In [202]:
output_exp2_results()

## Experiment 3
prompt with incorrect answers

In [175]:
incorrect_answers = [a['answer'] for a in updated_answers if not a['correct']]

In [176]:
len(incorrect_answers)

66

In [177]:
def generate_exp3_prompt(n=5):
    preamble = """Given a math problem, generate an answer with a rationale.
    
Examples:
    """
    lines = []
    lines.append(preamble)
    examples = random.sample(incorrect_answers, n)
    for example in examples:
        lines.append(f'Question: {question}\n')
        lines.append(example)
        lines.append('\n')
    lines.append(f'Question: {question}\n')
    return('\n'.join(lines))

In [179]:
print(generate_exp3_prompt())

Given a math problem, generate an answer with a rationale.
    
Examples:
    
Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?

The increase in value added 80,000*.15=$<<80000*.15=12000>>12,000 to its value
So the house is worth 80,000+12,000=$<<80000+12000=92000>>92,000 after the repairs
So he earned a profit of 92,000-80,000=$<<92000-80000=12000>>12,000. Answer: \boxed{12000}.


Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?

The increase in price was 80,000*.15=$<<80000*.15=12000>>12,000
So the house is worth 80,000+12,000=$<<80000+12000=92000>>92,000
So he made a profit of 92,000-80,000=$<<92000-80000=12000>>12,000. Answer: \boxed{12,000}.


Question: Josh decides to try flipping a house.  He buys 

In [181]:
def exp3(trials=100):
    for i in range(trials):
        print(f'{i})'.rjust(4, ' '), end='')
        prompt = generate_exp3_prompt()
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                 "content": prompt},
            ]
        )
        answer = response.choices[0]['message']['content']
        correct = is_correct(answer)
        exp3_answers.append({'answer': answer,
                            'correct': correct})
        if correct:
            print('correct')
        else:
            print('INCORRECT')

In [182]:
exp3_answers = []
start = datetime.now()
print(f'Start: {start}')
exp3()
end = datetime.now()
print(f'End: {end}')
elapsed = end - start
print(f'Elapsed: {elapsed}')

Start: 2023-10-28 16:36:39.978297
  0)correct
  1)correct
  2)INCORRECT
  3)INCORRECT
  4)INCORRECT
  5)correct
  6)correct
  7)INCORRECT
  8)INCORRECT
  9)INCORRECT
 10)INCORRECT
 11)correct
 12)correct
 13)correct
 14)correct
 15)correct
 16)INCORRECT
 17)correct
 18)INCORRECT
 19)correct
 20)correct
 21)INCORRECT
 22)correct
 23)correct
 24)INCORRECT
 25)correct
 26)correct
 27)INCORRECT
 28)correct
 29)INCORRECT
 30)INCORRECT
 31)correct
 32)correct
 33)correct
 34)INCORRECT
 35)INCORRECT
 36)INCORRECT
 37)INCORRECT
 38)correct
 39)INCORRECT
 40)INCORRECT
 41)INCORRECT
 42)INCORRECT
 43)INCORRECT
 44)correct
 45)correct
 46)INCORRECT
 47)INCORRECT
 48)correct
 49)INCORRECT
 50)INCORRECT
 51)INCORRECT
 52)correct
 53)correct
 54)INCORRECT
 55)correct
 56)correct
 57)INCORRECT
 58)INCORRECT
 59)INCORRECT
 60)correct
 61)INCORRECT
 62)INCORRECT
 63)INCORRECT
 64)INCORRECT
 65)INCORRECT
 66)INCORRECT
 67)INCORRECT
 68)correct
 69)correct
 70)INCORRECT
 71)correct
 72)INCORRECT
 73)INCO

In [183]:
len(exp3_answers)

100

In [188]:
def output_exp3_results():
    basedir = 'data/results/exp3'
    os.makedirs(basedir, exist_ok=True)    
    correct_counter = Counter([a['correct'] for a in exp3_answers])
    accuracy = correct_counter[True] / (correct_counter[True] + correct_counter[False])
    with open(f'{basedir}/exp3_master.txt', 'w') as file:
        file.write(f'Accuracy: {accuracy}\n')
        file.write(f'Question: {question}\n\n')
        file.write(f'Correct Answers:\n===============\n')
        for answer in exp3_answers:
            if answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')
                
        file.write(f'\n\n\nIncorrect Answers:\n===============\n')
        for answer in exp3_answers:
            if not answer['correct']:
                file.write(answer['answer'])
                file.write('\n\n')

In [189]:
output_exp3_results()